### Import Prerequis

In [1]:
import sys
import os
import subprocess

sys.path.append(os.path.abspath(os.path.join('..')))

### Compile Library C and generate JSON (make)

In [2]:
try:
    result = subprocess.run(
        "make -C ../libc",
        shell=True,
        capture_output=True,
        text=True
    )
    print(result.stdout)

    if result.stderr:
        print(result.stderr)
    
    if result.returncode != 0:
        print(f"Build failed with exit code {result.returncode}")
        sys.exit(1)

except Exception as e:
    print(f"Build failed: {e}")
    sys.exit(1)

make: Entering directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/ArtGenre-Classifier/libc'
OS        : Windows
Arch      : AMD64
Extension : .dll
Compiler  : gcc
Python    : /mingw64/bin/python3
  GEN     JSON specs
Processing: src\array.c
Processing: src\linearModel.c
Processing: src\math.c
Processing: src\utils.c
Successfully generated 4 JSON files in 'specs'
make: Leaving directory '/c/Users/milha/Documents/ESGI/3ESGI CL A ALT RO/PA/ArtGenre-Classifier/libc'



### Import Loader

In [3]:
from engine.interop.loader import Loader

# On doit restart kernel si on veut recharger la lib
try:
    Loader.loadLibrary(
        lib_name="libc",
        lib_folder="../libc",
        build_folder="../libc/build",
        specs_folder="../libc/specs"
    )

except Exception as e:
    if "already loaded" not in str(e).lower():
        raise RuntimeError(f"Failed to load library: {e}")
    
    print("Library already loaded, skipping.")

### Import Interop

In [4]:
import engine.interop.math as Math
from engine.interop.array import ArrayFloat32Ptr

### Tests (deja fait dans le pytest)

In [5]:
"""Une length > uint32_max doit lever une ValueError"""

try:
    ArrayFloat32Ptr.init_from_incrementing_numbers(5_000_000_000)
except ValueError as e:
    print(f"Expected error for length > uint32_max:\n\n{e}")

Expected error for length > uint32_max:

ArrayFloat32Ptr.init_from_incrementing_numbers(): _LibLoader.check_ctype(): value 5000000000 out of bounds: 0 to 4294967295 (uint32)
